In [1]:
import numpy as np
import scipy as sci
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
from math import isnan
from sklearn import tree

In [2]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]
for i in range(len(tableau20)):
    r, g, b = tableau20[i]
    tableau20[i] = (r / 255., g / 255., b / 255.)

In [3]:
class H1bInfo():


    #load the data from path
    def read_process_data(self,dataPath):
        H1B = pd.read_csv(dataPath)
        return H1B


    #Analyze the H1Bs by the status of their visa applications and show the plot
    def showCASE_STATUS(self,H1B):
        statusCount = H1B['CASE_STATUS'].value_counts()
        statusTypes = statusCount.index.copy(deep=True)
        statusTypes = statusTypes.values
        statusTypes[4] = 'Others'   # PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED , simplified to 'other'
        statusTypes = statusTypes[0:5]

        statusValues = statusCount.copy()
        statusValues = statusValues.values
        statusValues[4] = np.sum(statusValues[4:7])  # sum all of the others
        statusValues = statusValues[0:5]


        piefig = plt.pie(statusValues, autopct='%1.00f%%', shadow=False, startangle=90)
        plt.legend(statusTypes, loc="best", prop={'size':6})
        plt.axis('equal')
        plt.tight_layout()
        plt.title('Case Status percentage')
        plt.show()


    #K-Means Clustering to seperate the h1b location
    def K_meanAnalyze(self,H1B):
        #Get the states values
        H1City,H1State = H1B['WORKSITE'].str.split(', ',1).str
        #Reading Latitude and Longitude data (taking out null)
        H1LatLong = H1B.loc[H1State != 'NA']
        H1LatLong = H1LatLong[['lon','lat']]
        H1LatLong = H1LatLong.dropna()
        H1Long = H1LatLong['lon'].values
        H1Lat = H1LatLong['lat'].values

        #K-Means clustering to see where the applicants were
        nClusters = 10
        kmeans = KMeans(n_clusters=nClusters, random_state=0).fit(H1LatLong.values)
        #Getting the cluster for each case
        kl = kmeans.labels_
        H1LatLong['Cluster'] = kl
        H1LatLong['State'] = H1State
        H1LatLong = H1LatLong.dropna()
        return H1LatLong

    #show the plot of the h1b WORKSITE after k-mean
    #dense is the parameter to control distance between point, when gainning the info from the H1LatLong
    def showWORKSITE(self,dense,H1LatLong):


        #dense = 2000/1000 (usually )
        pltIndex = range(0,2892144,dense)

        #plot collor
        colorSet = ['ro','go','bo','co','mo','ko','r*','g*','b*','c*']
        CIndex = range(0,10)

        for i in CIndex:
            plt.plot(H1LatLong.iloc[pltIndex,0].loc[H1LatLong.iloc[pltIndex,2] == i], H1LatLong.iloc[pltIndex,1].loc[H1LatLong.iloc[pltIndex,2] == i], colorSet[i])

        #axis label name
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.legend(CIndex, loc="upper left", prop={'size':12})
        plt.title('H1b application distribution')
        plt.show()

    #Salary Data according to clusters
    def salaryAnalyze(self,H1B,H1LatLong):

        nClusters = 10
        H1Salary = H1LatLong
        H1Salary['Salary'] = H1B['PREVAILING_WAGE']
        H1Salary['FT'] = H1B['FULL_TIME_POSITION']

        salaryMin = np.zeros(nClusters)
        salaryMax = np.zeros(nClusters)
        salaryMean = np.zeros(nClusters)
        salaryMedian = np.zeros(nClusters)
        salaryStd = np.zeros(nClusters)

        nClusters = 10
        for k in range(0,nClusters):
            #which cluster belongs to
            salaryClustered = H1Salary.loc[H1Salary['Cluster'] == k]
            #only consider full-time employment
            salaryClustered = salaryClustered.loc[salaryClustered['FT'] == 'Y']
            #take out extreme outliers
            salaryClustered = salaryClustered.loc[salaryClustered['Salary'] <= 5000000]
            # take out non-paid positions
            salaryClustered = salaryClustered.loc[salaryClustered['Salary'] > 0]
            #drop N/A
            salaryClustered = salaryClustered.dropna()

            #each cluster's description
            salaryMin[k] = np.min(salaryClustered['Salary'].values)
            salaryMax[k] = np.max(salaryClustered['Salary'].values)
            salaryMean[k] = np.mean(salaryClustered['Salary'].values)
            salaryMedian[k] = np.median(salaryClustered['Salary'].values)
            salaryStd[k] = np.std(salaryClustered['Salary'].values)
        return [salaryMin,salaryMax,salaryMean,salaryMedian,salaryStd]

    #show the salary detail after k-mean process
    def showSALARY_table(self,salaryMin,salaryMax,salaryMean,salaryMedian,salaryStd):
         #build a DataFrame to show.
        salaryStats = pd.DataFrame()
        salaryStats['Mean'] = salaryMean.astype(int)
        salaryStats['Median'] = salaryMedian.astype(int)
        salaryStats['Std'] = salaryStd.astype(int)
        salaryStats['Min'] = salaryMin.astype(int)
        salaryStats['Max'] = salaryMax

        print(salaryStats)

    #Plotting and comparison to the median US salary (2015)

    def showSALARY_plot(self,salaryMedian,salaryMean):
        colorSet = ['r','g','b','c','m','k','r','g','b','c']
        colorSetS = ['r*','g*','b*','c*','m*','k*','r*','g*','b*','c*']
        CIndex = range(0,10)
        for i in CIndex:
            plt.bar(i,salaryMedian[i], color = colorSet[i])

        for i in CIndex:
            plt.plot(i,salaryMean[i], colorSet[i])

        plt.plot(np.arange(-0.5,10.5,1),55775*np.ones(11), "b--")
        plt.xlabel('Cluster number.')
        plt.ylabel('Median Salary (USD)')
        plt.title('Salary for every work part')
        plt.show()

    #show top 6 company who has apply to the h1b for employee
    def showTOP6com_table(self,H1B):
        comptable = H1B['EMPLOYER_NAME'].value_counts().sort_values(ascending=False).head(6)
        comptable.plot(kind='barh')
        plt.ylabel('H1B number')
        plt.xlabel('company_name')
        plt.title('Top6 company application number')

        comptable2=H1B[H1B['EMPLOYER_NAME'].isin(comptable.index.values)]
        comptable2 = comptable2.groupby(['EMPLOYER_NAME','YEAR']).size().unstack()
        comptable2.plot(kind='barh')
        plt.ylabel('H1B number')
        plt.xlabel('company_name')
        plt.legend(loc="upper right", prop={'size':8})
        plt.title('Company history application number')
        plt.show()

    #show the h1b number in everyear's change
    def showYearTrend_plot(self,H1B):
        yearTrend = H1B['YEAR'].value_counts().sort_values(ascending=True)
        yearTrend.plot(kind = 'bar')
        plt.title('Recent year for H1B application number')
        plt.show()

    #show top20 popular Jobtitle and top10 Worksites for H1-B Visa holders
    def showJOBTITLE_plot(self,H1B):
        H1B['JOB_TITLE'].value_counts().sort_values(ascending=False).head(20).plot(kind='barh',color=tableau20)
        plt.title('Top10 jobs for h1b application')
        plt.show()
        H1B['WORKSITE'].value_counts().head(10).plot(kind='barh',color=tableau20)
        plt.title('Top10 cities for h1b application')
        plt.show()

    #show top20 salary mean Jobtitle
    def showAVGSalary_plot(self,H1B):
        avgWagePerJob = H1B.groupby(['JOB_TITLE']).mean()['PREVAILING_WAGE'].nlargest(20).plot(kind="barh",color=tableau20)
        plt.title('Salary for Top20 jobs')
        plt.show()

    #show the difference between fulltime job and part time job.
    def showFullvsPart_plot(self,H1B):
        fullTime = H1B.FULL_TIME_POSITION.value_counts().plot(kind = 'bar',color=[(0.2,0.8,0.2),(0.8,0.2,0.2)])
        plt.ylabel('H1B number')
        plt.xlabel('JOb Type')
        plt.legend(loc="upper right", prop={'size':8})
        plt.title('Full time job VS Part-time job')
        plt.show()

    # CASE_STATUS situation predict. using decision Tree
    def predict_decision_tree(self,H1B,job_TitleName,WORKSITE,EMPLOYER_NAME,PREVAILING_WAGE,dense):

        #top10JobList = H1B['JOB_TITLE'].value_counts().sort_values(ascending=False).head(20).index.values
        top10JobDB = H1B[H1B['JOB_TITLE']==job_TitleName]
        #remove 2016, use other years as training day
        top10JobDB1 = top10JobDB[top10JobDB['YEAR'] != 2016]
        sampler = np.random.permutation(top10JobDB1.count()/dense)
        top10JobDB1 = top10JobDB1.take(sampler)


        #-----------  data process  -----------
        xSet1 = top10JobDB1.loc[:,'WORKSITE']
        WORKSITE_List = xSet1.drop_duplicates().values
        X_input1 = []
        for i in xSet1:
            X_input1.append(WORKSITE_List.tolist().index(i))

        xSet2 = top10JobDB1.loc[:,'EMPLOYER_NAME']
        EMPLOYER_NAME_List = xSet2.drop_duplicates().values
        X_input2 = []
        for i in xSet2:
            X_input2.append(EMPLOYER_NAME_List.tolist().index(i))

        xSet3 = top10JobDB1.loc[:,'CASE_STATUS']
        CASE_STATUS_List = xSet3.drop_duplicates().values
        X_input3 = []
        for i in xSet3:
            X_input3.append(CASE_STATUS_List.tolist().index(i))

        #salary range
        xSet4 = top10JobDB1.loc[:,'PREVAILING_WAGE']
        #define salary range
        salaryRange = [20000.00,40000.00,60000.00,80000.00,100000.00,120000.00,140000.00,
                       160000.00,180000.00,200000.00,400000.00,500000.00,1000000.00,2000000.00,float("inf")]
        X_input4 = []
        count = 0
        for i in xSet4:
            # deal with na !
            if isnan(i):
                i = 0
            for j in range(0,len(salaryRange)):
                if i <= salaryRange[j]:
                    X_input4.append(j)
                    break
                elif i > salaryRange[j]:
                    continue


        # -----------  decision Tree ALG  -----------
        x_input = np.zeros([3,len(X_input4)])
        x_input[0] = X_input1
        x_input[1] = X_input2
        x_input[2] = X_input4
        x_matrix = x_input.transpose()

        clf = tree.DecisionTreeClassifier()
        clf = clf.fit(x_matrix, X_input3)
        #predict
        salary_level = 0
        for j in range(0,len(salaryRange)):
                if PREVAILING_WAGE <= salaryRange[j]:
                    salary_level = j
                    break
                elif i > salaryRange[j]:
                    continue

        if WORKSITE in WORKSITE_List.tolist():
            worksite = WORKSITE_List.tolist().index(WORKSITE)
        else:
            worksite = 0
        if EMPLOYER_NAME in EMPLOYER_NAME_List.tolist():
            employ = EMPLOYER_NAME_List.tolist().index(EMPLOYER_NAME)
        else:
            employ = 0


        tree_result = clf.predict([[worksite, employ,salary_level ]])

        return CASE_STATUS_List[tree_result]


    # test one specify job's accuracy in 2016 base on 2011-2015 data
    def test_prediction_accuracy(self,H1B,job_TitleName,dense=10):
        top10JobDB = H1B[H1B['JOB_TITLE']==job_TitleName]
        # 2016data
        top10JobDB1 = top10JobDB[top10JobDB['YEAR'] == 2016]
        num_of_row = len(top10JobDB1.index)

        accuracy_count = 0
        for i in range(0,num_of_row,dense):
            work_site = top10JobDB1[i:i+1]['WORKSITE'].values
            emp_name = top10JobDB1[i:i+1]['EMPLOYER_NAME'].values
            wage = top10JobDB1[i:i+1]['PREVAILING_WAGE'].values
            if self.predict_decision_tree(top10JobDB,job_TitleName,work_site,emp_name,wage,dense) == top10JobDB1[i:i+1]['CASE_STATUS'].values:
                accuracy_count +=1

        #print('accuracy_count')
        #print(accuracy_count)
        if num_of_row == 0:

            return 'no job data'
        else:

            #print(num_of_row)
            return float(accuracy_count)/num_of_row*dense

    #get top10 popular job's accuracy in 2016 base on 2011-2015
    def testTOP10JOB_acuracy(self,H1B):
        top10 = ['COMPUTER PROGRAMMER','SYSTEMS ANALYST','SOFTWARE DEVELOPER','BUSINESS ANALYST','COMPUTER SYSTEMS ANALYST',
                'TECHNOLOGY LEAD - US','SENIOR SOFTWARE ENGINEER','TECHNOLOGY ANALYST - US','ASSISTANT PROFESSOR','SENIOR CONSULTANT']



        dense = 10

        accuracy_rate = []
        for item in top10:
            accuracy_rate.append(self.test_prediction_accuracy(H1B,item,dense))


        result = dict(zip(top10,accuracy_rate))



        print(result)

        return result

In [4]:
h1bObj = H1bInfo()
try:
    H1B = h1bObj.read_process_data('h1b_kaggle.csv')
except IOError:
    print("Error: cannot find the path,pleas put data and program in the same folder")
else:
    print("data file read successfully")

data file read successfully


In [5]:
k_meanResult = h1bObj.K_meanAnalyze(H1B)
salary_Result = h1bObj.salaryAnalyze(H1B,k_meanResult)

In [ ]:
h1bObj.showWORKSITE(H1B,10)

In [ ]:
h1bObj.showCASE_STATUS(H1B)

In [ ]:
h1bObj.showTOP6com_table(H1B)

In [ ]:
h1bObj.showYearTrend_plot(H1B)

In [ ]:
h1bObj.showJOBTITLE_plot(H1B)

In [ ]:
h1bObj.showAVGSalary_plot(H1B)

In [ ]:
#=============================================
#K-means and Decision Tree

In [6]:
h1bObj.showSALARY_table(salary_Result[0],salary_Result[1],salary_Result[2],salary_Result[3],salary_Result[4])

    Mean  Median    Std    Min        Max
0  72241   67579  35587   2012  4865100.0
1  68792   64563  38669   2011  4648800.0
2  67338   62171  34323  15080  4180800.0
3  68359   63981  34810  14000  4996888.0
4  65174   61734  28070  15226  4538560.0
5  84536   80371  34519   2000  4985760.0
6  63692   58157  48857  15080  4848792.0
7  61082   55515  37206  16827  1040000.0
8  80774   81286  27908  14851  2897400.0
9  67026   63107  36651     63  4253600.0


In [ ]:
h1bObj.showSALARY_plot(salary_Result[3],salary_Result[2])

In [ ]:
h1bObj.showWORKSITE(50,k_meanResult)

In [47]:
job_title = 'SOFTWARE DEVELOPER'
worksite = 'SAN JOSE, CALIFORNIA'
emp_name = 'FACEBOOK'
wage = 180000.0
out = h1bObj.predict_decision_tree(H1B, job_title, worksite, emp_name, wage, dense=10)
print('Predicted Case Status:')
print(out)

Predicted Case Status:
['CERTIFIED']


In [55]:
job_title = 'PROGRAMMER'
worksite = 'BIRMINGHAM, ALABAMA'
emp_name = 'WHATEVER INC'
wage = 10000.0
out = h1bObj.predict_decision_tree(H1B, job_title, worksite, emp_name, wage, dense=10)
print('Predicted Case Status:')
print(out)

Predicted Case Status:
['CERTIFIED-WITHDRAWN']


In [59]:
job_title = 'PROGRAMMER'
out = h1bObj.test_prediction_accuracy(H1B, job_title)
print('Accuracy rate:')
print(out)

Accuracy rate:
0.2857142857142857


In [ ]:
h1bObj.testTOP10JOB_acuracy(H1B)

In [66]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from datetime import datetime

def get_data(path_to_data, chunk='all'):
    if chunk == 'all':
        dwdata = pd.read_csv(path_to_data)
    else:
        dwdata = pd.read_csv(path_to_data, nrows = chunk)
        
    return dwdata

def format_clean(data):
    data2 = data.drop('Unnamed: 0', axis = 1)
    df1 = data2[data2['CASE_STATUS']!='WITHDRAWN']
    df2 =df1[df1['CASE_STATUS']!='CERTIFIED-WITHDRAWN']
    
    df3 = pd.get_dummies(df2.drop('CASE_STATUS', axis = 1))
    df4 = pd.concat([df3, df2['CASE_STATUS']], axis=1)
    df5 = df4.dropna()
    
    return df5


data_set_path = 'h1b_kaggle.csv'

chunks = 10000
data_set = get_data(data_set_path, chunk = chunks)

new_data = format_clean(data_set)

xs = new_data.drop('CASE_STATUS', axis = 1)  
ys = new_data['CASE_STATUS']
X_train, X_test, y_train, y_test = train_test_split(xs,ys, test_size=0.30)

scaler = preprocessing.StandardScaler()
scaler.fit(X_train)

LR = LogisticRegression()
LR.fit(scaler.transform(X_train), y_train)
y_pred=LR.predict(scaler.transform(X_test))

In [67]:
print("Accuracy: {}".format(accuracy_score(y_test,y_pred)))

Accuracy: 0.9428007889546351
